In [2]:
#!pip install adversarial-robustness-toolbox

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score
import sklearn.linear_model #import LogisticRegression

In [ ]:
# ART imports
from art import config
from art.utils import load_dataset, get_file
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, ZooAttack, ProjectedGradientDescent
from art.attacks.evasion import BasicIterativeMethod, DeepFool, CarliniLInfMethod
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification.scikitlearn import ScikitlearnLogisticRegression
from art.defences.detector.evasion import BinaryInputDetector

In [ ]:
#Implementation of logistic Regression
#from LogisticRegression import LogisticRegression
from accuracy import Accurator

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('../data/heart.csv')
df.head()

# DATA PRE PROCESSING

**One hot encoding**

In [ ]:
# categorical columns
cp = pd.get_dummies(df['cp'],prefix= 'cp')
thal = pd.get_dummies(df['thal'],prefix = 'thal')
slope = pd.get_dummies(df['slope'],prefix = 'slope')

#merge dummy variables to main data frame
frames = [ df,cp,thal,slope]
df = pd.concat(frames,axis=1)

#Dropping the unnecessory variables
df=df.drop(columns = ['cp','thal','slope'])
df.head()

**Separating Target Variable**

In [ ]:
#Declaring Dependent and Independent variables
X = df.drop(['target'],axis=1)
y = df['target']

**Data Normalization**

In [ ]:
X = MinMaxScaler().fit_transform(X)

**Splitting Training and Testing Dataset**

In [ ]:
#Splitting x and y into training and testing datasets

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=4)

In [ ]:
#checking the shape of X_train and X_test
X_train.shape, X_test.shape

# Training Model

In [ ]:
#model = LogisticRegression(learning_rate=0.1, num_iterations=300000)
estimator_lr=sklearn.linear_model.LogisticRegression(random_state=0)

In [ ]:
%%time
#model.fit(X_train, y_train)
estimator_lr.fit(X_train,y_train)  # about 3min

# Predictions

In [ ]:
preds = estimator_lr.predict(X_test)
accurator=Accurator(y_test)

In [ ]:
accurator.print_report(preds)

In [ ]:
accurator.print_confusionMatrix(preds)

# Using ATTACKS

In [ ]:
art_regressor = ScikitlearnLogisticRegression(estimator_lr)

In [ ]:
attacker = FastGradientMethod(art_regressor, eps=0.5)
x_test_adv = attacker.generate(X_test[:100])

In [ ]:
x_test_adv_pred = estimator_lr.predict(x_test_adv)
accurator_atk=Accurator(y_test[:100])
accurator_atk.print_confusionMatrix(x_test_adv_pred)

# Using defense

In [ ]:
detector_lr=sklearn.linear_model.LogisticRegression(random_state=0)
detector = BinaryInputDetector(detector_lr)